In [1]:
# Identificação facial

In [2]:
import os
import numpy as np
import cv2
import face_recognition
from sklearn import svm

In [3]:
import statistics
import math

In [4]:
diretorio_trabalho = '/home/pi/Documents/Identificação facial/Indivíduos/'
os.chdir(diretorio_trabalho)

In [5]:
class Pessoa:
    def __init__(self, nome='Não definido'):
        self.nome = nome
        self.minuncias = []

    def calculaMinuncia(self, endereco_imagem):  
        imagem = face_recognition.load_image_file(endereco_imagem)
        minuncia = face_recognition.face_encodings(imagem)[0]
        self.minuncias.append(minuncia)     
    
    def minunciaMatch(self, face_minuncias):
        dist = []
        for aux_minuncias in self.minuncias:
            dist.append(np.linalg.norm(face_minuncias-aux_minuncias))
        mediana = statistics.median(dist)      
        return mediana

In [6]:
# Cria "Jaime".
jaime = Pessoa('Jaime')
jaime.calculaMinuncia(diretorio_trabalho+'Jaime/WhatsApp Image 2021-02-10 at 10.59.35.jpeg')
jaime.calculaMinuncia(diretorio_trabalho+'Jaime/WhatsApp Image 2021-02-10 at 10.59.36.jpeg')
jaime.calculaMinuncia(diretorio_trabalho+'Jaime/WhatsApp Image 2021-02-10 at 10.59.36 (1).jpeg')
jaime.calculaMinuncia(diretorio_trabalho+'Jaime/WhatsApp Image 2021-02-10 at 10.59.37.jpeg')
jaime.calculaMinuncia(diretorio_trabalho+'Jaime/WhatsApp Image 2021-02-10 at 10.59.37 (1).jpeg')

In [7]:
# Cria "Obama".
obama = Pessoa('Obama')
obama.calculaMinuncia(diretorio_trabalho+'Obama/obama.jpg')
obama.calculaMinuncia(diretorio_trabalho+'Obama/obama2.jpg')

In [8]:
# Cria "Priscila".
priscila = Pessoa('Priscila')
priscila.calculaMinuncia(diretorio_trabalho+'Priscila/WhatsApp Image 2021-02-10 at 10.58.17.jpeg')
priscila.calculaMinuncia(diretorio_trabalho+'Priscila/WhatsApp Image 2021-02-10 at 10.58.18.jpeg')
priscila.calculaMinuncia(diretorio_trabalho+'Priscila/WhatsApp Image 2021-02-10 at 10.58.18 (1).jpeg')
priscila.calculaMinuncia(diretorio_trabalho+'Priscila/WhatsApp Image 2021-02-10 at 10.58.19.jpeg')

In [9]:
# Calcula as minuncias de uma imagem qualquer.
endereco_imagem = diretorio_trabalho+'Obama2.jpg'
imagem = face_recognition.load_image_file(endereco_imagem)
faces_locacoes = face_recognition.face_locations(imagem)
face_minuncias = None
n_faces = len(faces_locacoes)
if n_faces==1:
    print("Face detectada.")
    face_minuncias = face_recognition.face_encodings(imagem)[0]
elif n_faces==0:
    print("Nenhuma face detectada!")
else:
    print("Mais de uma face detectada!")       

Face detectada.


In [10]:
# Obtem o match simples.
print(obama.minunciaMatch(face_minuncias))

0.20772843915292652


In [11]:
print(jaime.minunciaMatch(face_minuncias))

0.7213100677673254


In [12]:
print(priscila.minunciaMatch(face_minuncias))

0.808541783093114


In [13]:
class grupoPessoas:
    def __init__(self, nome='Não definido'):
        self.nome = nome
        self.pessoas = []
    
    def inserePessoa(self, pessoa):
        self.pessoas.append(pessoa)
        
    def buscaPessoa(self, nome):
        pessoa = None
        for aux_pessoa in self.pessoas:
            if aux_pessoa.nome==nome:
                pessoa = aux_pessoa
        return pessoa              
        
    def mostraMinuncias(self):
        minuncias = []
        nomes = []
        for aux_pessoa in self.pessoas:
            minuncias = minuncias+aux_pessoa.minuncias
            nomes = nomes+[aux_pessoa.nome]*len(aux_pessoa.minuncias)
        return {'minuncias': minuncias, 'nomes': nomes}
    
    def calibraClassificador(self):
        self.clf = svm.SVC(gamma='scale')
        aux_minuncias = self.mostraMinuncias()
        self.clf.fit(aux_minuncias["minuncias"], aux_minuncias["nomes"])
        return self.clf.score(aux_minuncias["minuncias"], aux_minuncias["nomes"])
    
    def identificaPessoaSVM(self, face_minuncias):
        nome = self.clf.predict([face_minuncias])[0]
        escore = self.buscaPessoa(nome).minunciaMatch(face_minuncias)
        return {'nome': nome, 'escore':escore}
    
    def identificaPessoaMD(self, face_minuncias):
        nome = None
        distancia = math.inf
        for aux_pessoa in self.pessoas:
            aux_distancia = aux_pessoa.minunciaMatch(face_minuncias)
            if aux_distancia<distancia:
                distancia = aux_distancia
                nome = aux_pessoa.nome
        return {'nome': nome, 'distancia':distancia}
    
    

In [14]:
grupo_teste = grupoPessoas("teste")

In [15]:
grupo_teste.inserePessoa(jaime)
grupo_teste.inserePessoa(priscila)
grupo_teste.inserePessoa(obama)

In [16]:
grupo_teste.buscaPessoa('Jaime').nome

'Jaime'

In [17]:
grupo_teste.calibraClassificador()

0.8181818181818182

In [22]:
grupo_teste.identificaPessoaSVM(face_minuncias)

{'nome': 'Jaime', 'escore': 0.7213100677673254}

In [23]:
grupo_teste.identificaPessoaMD(face_minuncias)

{'nome': 'Obama', 'distancia': 0.20772843915292652}